In [3]:
import pandas as pd
import json
import gzip
from zipfile import ZipFile
import mysql.connector
import os

In [ ]:
#  mol1 and mol2: 139 rows
SELECT a.curated_epitope_id, b.object_id, b.starting_position, b.ending_position, b.mol1_seq, 
    b.mol1_source_id, b.mol2_source_id
FROM curated_epitope a, object b
WHERE a.e_object_id = b.object_id AND b.starting_position is NOT NULL 
    AND b.mol1_source_id is NOT NULL and b.mol2_source_id is NOT NULL
limit 10;

In [294]:
# mol1: 139 
SELECT a.curated_epitope_id, b.object_id, b.starting_position, b.ending_position,
    b.mol1_seq, b.mol1_source_id
FROM curated_epitope a, object b
WHERE a.e_object_id = b.object_id AND b.starting_position is NOT NULL 
    AND b.mol1_source_id is NOT NULL;

# mol2: 3553565
SELECT a.curated_epitope_id, b.object_id, b.starting_position, b.ending_position,
    b.mol1_seq, b.mol1_source_id, b.mol2_modified_seq, b.mol2_source_id
FROM curated_epitope a, object b
WHERE a.e_object_id = b.object_id
    AND b.starting_position is NOT NULL 
    AND b.mol2_source_id is NOT NULL
LIMIT 10;

In [ ]:
# show
SELECT a.curated_epitope_id, a.e_name, e_region_domain_flag,
    b.object_id, b.starting_position, b.ending_position, b.mol1_seq,
    c.accession, c.database, c.name, c.sequence
FROM curated_epitope a, object b, source c
WHERE a.e_object_id = b.object_id
    AND b.mol2_source_id = c.source_id
    AND b.starting_position is NOT NULL
    AND c.sequence is NOT NULL
LIMIT 10;

# count rows
SELECT COUNT(DISTINCT a.curated_epitope_id)
FROM curated_epitope a, object b, source c
WHERE a.e_object_id = b.object_id
    AND b.mol2_source_id = c.source_id
    AND b.starting_position is NOT NULL;

# counts antigens with epitopes
SELECT COUNT(DISTINCT c.accession)
FROM curated_epitope a, object b, source c
WHERE a.e_object_id = b.object_id
    AND b.mol2_source_id = c.source_id
    AND b.starting_position is NOT NULL;

In [26]:
db_params = {
    'host': "localhost",
    'user': "root",
    'passwd': "root",
    'database': 'protein',
}
conn = mysql.connector.connect(**db_params)

cursor = conn.cursor()
cursor.execute("""
        SELECT a.curated_epitope_id,
            b.starting_position, b.ending_position, b.mol1_seq,
            c.accession
        FROM curated_epitope a, object b, source c
        WHERE a.e_object_id = b.object_id
            AND b.mol2_source_id = c.source_id
            AND b.starting_position is NOT NULL
""")

In [ ]:
epitopes = {}
keys = ['epitope_id', 'start', 'end', 'seq', 'acc']
for row in cursor:
    row = [i if isinstance(i, str) else int(i) for i in row]
    acc = row[-1]
    if acc not in epitopes:
        epitopes[acc] = []
    epitopes[acc].append(row[:-1])
print(len(epitopes))    

In [20]:
import re


[1507426,
 '448',
 'Exact Epitope',
 1294668,
 157,
 171,
 'ASGLVALLLNSLVVI',
 'YP_001135796.1',
 'GenPept',
 'hypothetical protein',
 'MVNKAADRWSTGLPGLRSGMSTPMQGIGGLLSMSADAVKFLFRRPFQTSEFLLQSWFVARVSLAPTLLVAIPFTVLVSFTLNILLSELGAADLSGAGAAFGAVTQVGPMVTVLIVAGAGATAMCADLGSRTIREEIDALEVLGINPVQRLVTPRMLASGLVALLLNSLVVIIGIVGGYVFSIFVQGVNPGAFAAGITLLTGVPEVIISCVKAALFGLIAGLVACYRGLSISTGGAKAVGNAVNETVVYAFMALFVVNVVVTAIGIQMTSG']

In [19]:
type(row[1])

str

In [28]:
def read_json(local_file:str) -> list:
    # read json
    data_list = []
    with ZipFile(local_file) as zf:
        for file in zf.namelist():
            with zf.open(file) as f:
                data = json.load(f)
                data_list += data['Data']
    return data_list

infile = '/home/yuan/data/IEDB/epitope_full_v3_json.zip'
data = read_json(infile)

In [175]:
for k, v in data[0].items():
    print(f"{k:40}\t:{v}")

Epitope ID - IEDB IRI                   	:http://www.iedb.org/epitope/1
Epitope - Object Type                   	:Linear peptide
Epitope - Name                          	:AA + MCM(A1,A2)
Epitope - Modified Residue(s)           	:A1,A2
Epitope - Modifications                 	:Main chain modification
Epitope - Starting Position             	:200
Epitope - Ending Position               	:201
Epitope - IRI                           	:
Epitope - Synonyms                      	:
Epitope - Source Molecule               	:streptokinase, SKase
Epitope - Source Molecule IRI           	:http://www.ncbi.nlm.nih.gov/protein/AAB20743.1
Epitope - Molecule Parent               	:Streptokinase A
Epitope - Molecule Parent IRI           	:http://www.uniprot.org/uniprot/P10520
Epitope - Source Organism               	:Streptococcus pyogenes serotype M3 D58
Epitope - Source Organism IRI           	:https://ontology.iedb.org/ontology/ONTIE_0000450
Epitope - Species                       	:Streptococcus pyo

In [54]:
# drop table
def drop_table(db_params:str, table_name:str):
    conn = mysql.connector.connect(**db_params)
    cursor = conn.cursor()
    tag = True
    try:
        cursor.execute(f"DROP TABLE {table_name}")
    except Exception as e:
        print(e)
        tag = False
    finally:
        conn.close()
    return tag
    
db_params = {
    'host': "localhost",
    'user': "root",
    'passwd': "root",
    'database': 'protein',
}
table_name = 'epitope'
res = drop_table(db_params, table_name)
print(res)

True


In [180]:
# estimate length
max_len = {k:0 for k in data[0]}
for el in data:
    for k, v in el.items():
        if len(v)> max_len[k]:
            max_len[k] = len(v)
for k, v in max_len.items():
    print(f"{k:20}:{v}")

Epitope ID - IEDB IRI:35
Epitope - Object Type:36
Epitope - Name      :829
Epitope - Modified Residue(s):111
Epitope - Modifications:50
Epitope - Starting Position:5
Epitope - Ending Position:5
Epitope - IRI       :43
Epitope - Synonyms  :4579
Epitope - Source Molecule:433
Epitope - Source Molecule IRI:51
Epitope - Molecule Parent:188
Epitope - Molecule Parent IRI:43
Epitope - Source Organism:95
Epitope - Source Organism IRI:48
Epitope - Species   :53
Epitope - Species IRI:48
Related Object - Epitope Relation:23
Related Object - Object Type:43
Related Object - Name:1532
Related Object - Starting Position:5
Related Object - Ending Position:5
Related Object - IRI:43
Related Object - Synonyms:1590
Related Object - Source Molecule:286
Related Object - Source Molecule IRI:51
Related Object - Molecule Parent:144
Related Object - Molecule Parent IRI:41
Related Object - Source Organism:124
Related Object - Source Organism IRI:48
Related Object - Species:52
Related Object - Species IRI:48


In [181]:
columns = [
        'Epitope_ID VARCHAR(100) PRIMARY KEY',
        'Epitope_IRI VARCHAR(200)',
        'Object_Type VARCHAR(50)',
        'Name VARCHAR(1000)',
        'Modified_Residue VARCHAR(150)',
        'Modifications VARCHAR(100)',
        'Starting_Position INT',
        'Ending_Position INT',
        'IRI VARCHAR(50)',
        'Synonyms VARCHAR(5000)',
        'Source_Molecule VARCHAR(500)',
        'Source_Molecule_IRI VARCHAR(100)',
        'Molecule_Parent VARCHAR(200)',
        'Molecule_Parent_IRI VARCHAR(100)',
        'Source_Organism VARCHAR(200)',
        'Source_Organism_IRI VARCHAR(200)',
        'Species VARCHAR(100)',
        'Species_IRI VARCHAR(100)',
]

In [185]:
# create table
def create_table(db_params:str, sql_query:str):
    conn = mysql.connector.connect(**db_params)
    cursor = conn.cursor()
    tag = True
    try:
        cursor.execute(sql_query)
    except Exception as e:
        print(e)
        tag = False
    finally:
        conn.close()
    return tag
    
db_params = {
    'host': "localhost",
    'user': "root",
    'passwd': "root",
    'database': 'protein',
}
table_name = 'epitope'

sql_query = f"CREATE TABLE {table_name} ({','.join(columns)})"
res = create_table(db_params, sql_query)
print(res)

True


In [210]:
rows = []
for el in data:
    item = list(el.values())[:17]
    # item = [i if i else None for i in item]
    id = os.path.basename(item[0])
    item.insert(0, id)
    rows.append(item)
print(len(rows), rows[0])

2236826 ['1', 'http://www.iedb.org/epitope/1', 'Linear peptide', 'AA + MCM(A1,A2)', 'A1,A2', 'Main chain modification', '200', '201', '', '', 'streptokinase, SKase', 'http://www.ncbi.nlm.nih.gov/protein/AAB20743.1', 'Streptokinase A', 'http://www.uniprot.org/uniprot/P10520', 'Streptococcus pyogenes serotype M3 D58', 'https://ontology.iedb.org/ontology/ONTIE_0000450', 'Streptococcus pyogenes', 'http://purl.obolibrary.org/obo/NCBITaxon_1314']


In [279]:
# insert rows into table
def insert_rows(db_params:str, table_name:str, columns:list, rows):
    conn = mysql.connector.connect(**db_params)
    cursor = conn.cursor()

    n, m = 0, 0
    col = ', '.join([i.split(' ')[0] for i in columns])
    for row in rows:
        try:
            val = []
            for a,b in zip(columns, row):
                _a = a.split(' ')
                if 'INT' == _a[1]:
                    val.append(b)
                else:
                    val.append(f"\"{b}\"")
            val = ", ".join(val)
            sql = (f"INSERT INTO epitope ({col}) VALUES ({val})")
            cursor.execute(sql, val)
            conn.commit()
            n += 1
        except Exception as e:
            print('error:', e)
            print([(a,b) for a,b in zip(columns, row)])
            m += 1
            break
    else:
        conn.close()
    return n, m
    
db_params = {
    'host': "localhost",
    'user': "root",
    'passwd': "root",
    'database': 'protein',
}
table_name = 'epitope'
res = insert_rows(db_params, table_name, columns, rows)
print(res)

error: Could not process parameters: str("1", "http://www.iedb.org/epitope/1", "Linear peptide", "AA + MCM(A1,A2)", "A1,A2", "Main chain modification", 200, 201, "", "", "streptokinase, SKase", "http://www.ncbi.nlm.nih.gov/protein/AAB20743.1", "Streptokinase A", "http://www.uniprot.org/uniprot/P10520", "Streptococcus pyogenes serotype M3 D58", "https://ontology.iedb.org/ontology/ONTIE_0000450", "Streptococcus pyogenes", "http://purl.obolibrary.org/obo/NCBITaxon_1314"), it must be of type list, tuple or dict
[('Epitope_ID VARCHAR(100) PRIMARY KEY', '1'), ('Epitope_IRI VARCHAR(200)', 'http://www.iedb.org/epitope/1'), ('Object_Type VARCHAR(50)', 'Linear peptide'), ('Name VARCHAR(1000)', 'AA + MCM(A1,A2)'), ('Modified_Residue VARCHAR(150)', 'A1,A2'), ('Modifications VARCHAR(100)', 'Main chain modification'), ('Starting_Position INT', '200'), ('Ending_Position INT', '201'), ('IRI VARCHAR(50)', ''), ('Synonyms VARCHAR(5000)', ''), ('Source_Molecule VARCHAR(500)', 'streptokinase, SKase'), ('S

In [104]:
print(len(columns), len(rows[0]))

17 17
